In [2]:
import os
import warnings
from pprint import pprint

In [3]:
import descarteslabs as dl

In [4]:
# *************FIND COUNTRY BOUNDARY ***************
matches = dl.places.find('burundi')
aoi = matches[0]
pprint(aoi)

# This area of interest just gives us some basic properties such as bounding boxes.
# To access a GeoJSON Geometry object of that place, we call the `Places.shape` method, in this case
# accessing a low-resolution version of this particular shape.
shape = dl.places.shape(aoi['slug'], geom='low')

{'bbox': [29.001302, -4.468538, 30.849959, -2.309987],
 'id': 85632285,
 'name': 'Burundi',
 'path': 'continent:africa_country:burundi',
 'placetype': 'country',
 'slug': 'africa_burundi'}


In [5]:
# import json

# *************FIND MATCHING SATELLITE IMAGES ***************
# PRODUCTS


# See what satellite products are available 
products = dl.metadata.available_products()
# pprint(products)

# isolate landsat 8 products
landsat_products = []
for product in products:
  if 'LC08'in product: 
    landsat_products.append(product)
# pprint(landsat_products)

feature_collection = dl.metadata.search(
    products='landsat:LC08:PRE:TOAR', 
    start_time='2014-03-12',
    end_time='2017-03-20',
    cloud_fraction= .2,
    place=aoi['slug'])

print (len(feature_collection['features']))

for f in feature_collection['features']:
    pprint('_________________')
    pprint(f['id'])
    pprint(f['properties']['acquired'])
    pprint(f['properties']['cloud_fraction'])

35
'_________________'
'landsat:LC08:PRE:TOAR:meta_LC81720622016148_v1'
'2016-05-27T08:07:56.623120Z'
0.1938
'_________________'
'landsat:LC08:PRE:TOAR:meta_LC81720622016164_v1'
'2016-06-12T08:07:59.899208Z'
0.0333
'_________________'
'landsat:LC08:PRE:TOAR:meta_LC81720622016180_v1'
'2016-06-28T08:08:07.544666Z'
0.0
'_________________'
'landsat:LC08:PRE:TOAR:meta_LC81720622016196_v1'
'2016-07-14T08:08:15.110594Z'
0.0253
'_________________'
'landsat:LC08:PRE:TOAR:meta_LC81720622016212_v1'
'2016-07-30T08:08:19.111433Z'
0.0013
'_________________'
'landsat:LC08:PRE:TOAR:meta_LC81720622016228_v1'
'2016-08-15T08:08:22.190113Z'
0.1318
'_________________'
'landsat:LC08:PRE:TOAR:meta_LC81720622016244_v1'
'2016-08-31T08:08:29.165524Z'
0.0003
'_________________'
'landsat:LC08:PRE:TOAR:meta_LC81720622016356_v1'
'2016-12-21T08:08:32.439796Z'
0.0841
'_________________'
'landsat:LC08:PRE:TOAR:meta_LC81720622017038_v1'
'2017-02-07T08:08:17.305499Z'
0.0682
'_________________'
'landsat:LC08:PRE:TOAR:met

In [8]:
# *************OUTPUT ARRAY ***************

# Rasterize the features.
#  * Select red, green, blue, alpha
#  * Scale the incoming data with range [0, 10000] down to [0, 4000] (40% TOAR)
#  * Choose an output type of "Byte" (uint8)
#  * Choose 60m resolution
#  * Apply a cutline of Burundi's boundary 
arr, meta = dl.raster.ndarray(
    "landsat:LC08:PRE:TOAR:meta_LC81720632016244_v1",
    bands=['red', 'green', 'blue', 'alpha'],
    scales=[[0,4000], [0, 4000], [0, 4000], None],
    data_type='Byte',
    resolution=60,
    cutline=shape['geometry']
)
'''
arr, meta = dl.raster.raster(
    "landsat:LC08:PRE:TOAR:meta_LC81720632016244_v1",
    bands=['red', 'green', 'blue', 'alpha'],
    scales=[[0,4000], [0, 4000], [0, 4000], None],
    data_type='Byte',
    resolution=60,
    cutline=shape['geometry'],
    save= "true"
)
'''

# Not sure if there is a dl equivalent, here are the results of gdalinfo via cli 

'''
Driver: GTiff/GeoTIFF
Files: landsat:LC08:PRE:TOAR:meta_LC81720632016244_v1_red-green-blue-alpha.tif
Size is 3430, 3985
Coordinate System is:
PROJCS["WGS 84 / UTM zone 36N",
    GEOGCS["WGS 84",
        DATUM["WGS_1984",
            SPHEROID["WGS 84",6378137,298.257223563,
                AUTHORITY["EPSG","7030"]],
            AUTHORITY["EPSG","6326"]],
        PRIMEM["Greenwich",0,
            AUTHORITY["EPSG","8901"]],
        UNIT["degree",0.0174532925199433,
            AUTHORITY["EPSG","9122"]],
        AUTHORITY["EPSG","4326"]],
    PROJECTION["Transverse_Mercator"],
    PARAMETER["latitude_of_origin",0],
    PARAMETER["central_meridian",33],
    PARAMETER["scale_factor",0.9996],
    PARAMETER["false_easting",500000],
    PARAMETER["false_northing",0],
    UNIT["metre",1,
        AUTHORITY["EPSG","9001"]],
    AXIS["Easting",EAST],
    AXIS["Northing",NORTH],
    AUTHORITY["EPSG","32636"]]
Origin = (55210.737599981250241,-255587.129183952696621)
Pixel Size = (60.000000000000000,-60.000000000000000)
'''


# Verify there are expected values in the array stack. 
num_rows = len(arr)
row_center =num_rows/2
print("Number of rows: ", num_rows)
print("Middle row: ", row_center)

num_cols = len(arr[0])
col_center = num_cols/2
print("Number of cols: ", num_cols)
print("Middle col: ", col_center)

middle_value = arr[row_center][col_center]

print("Middle value: ", middle_value)

# Note: A value of 1 in the alpha channel signifies where there is valid data.
# We use this throughout the majority of our imagery as a standard way of specifying
# valid or nodata regions. This is particularly helpful if a value of 0 in a particular
# band has meaning, rather than specifying a lack of data.

# *************TO DO, PLOT OR SAVE ALL IMAGES ***************
# *************FIND IDEAL TIME PERIOD FOR THIS DATASETS ***************
# Collect the id's for each feature
# ids = [f['id'] for f in feature_collection['features']]


Number of rows:  3985
Middle row:  1992.5
Number of cols:  3430
Middle col:  1715.0
Middle value:  [ 56  55  59 255]


/Users/karla/anaconda3/envs/poverty-predictor/lib/python3.5/site-packages/ipykernel_launcher.py:74: DeprecationWarning: using a non-integer number instead of an integer will result in an error in the future
/Users/karla/anaconda3/envs/poverty-predictor/lib/python3.5/site-packages/ipykernel_launcher.py:74: DeprecationWarning: using a non-integer number instead of an integer will result in an error in the future


In [ ]:
# *************PLOT SINGLE IMAGE ***************

import matplotlib.pyplot as plt
%matplotlib inline

plt.figure(figsize=[16,16])
plt.imshow(arr)

Next Steps:
1. Find and import Black Marble image into a different notebook. Note the year captured.
2. Test techniques on Sattelite and NightLight sample data. 
2. Complete the datasets - Identify all day Landsat 8 images over Burundi for the given Black Marble and output image from the year with less than 10% cloud cover.
3. Output the images, and select a subset that covers the country.